In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.getcwd()+"/../src/")
import nesse
import numpy as np

In [2]:
import time

import tracemalloc

from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass


def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break

        else:
            if not hasattr(o.__class__, '__slots__'):
                if hasattr(o, '__dict__'):
                    s+=sizeof(o.__dict__) # no __slots__ *usually* means a __dict__, but some special builtin classes (such as `type(None)`) have neither
                # else, `o` has no attributes at all, so sys.getsizeof() actually returned the correct value
            else:
                s+=sum(sizeof(getattr(o, x)) for x in o.__class__.__slots__ if hasattr(o, x))
        return s

    return sizeof(o) / 1024

import linecache

def display_top(snapshot, key_type='lineno', limit=10):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        print("#%s: %s:%s: %.1f KiB"
              % (index, frame.filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

In [43]:
tracemalloc.start()
ID = "4"
# print(tracemalloc.get_traced_memory())

print(os.getcwd())

events_filename = "g:/nesse/config/Events/e-_800keV_0inc.root"
Events = nesse.eventsFromG4root(events_filename)[:1]

# print("Events:")
# print(tracemalloc.get_traced_memory())

EF_filename = "g:/nesse/config/Fields/NessieEF_Base4e7Linear0-150.0V.hf"
WP_filename = "g:/nesse/config/Fields/NessieWP_4e7Linear0-150V_grid.hf"


Efield=nesse.fieldFromH5(EF_filename, rotate90=True)  
# print("EField:")
# print(tracemalloc.get_traced_memory())


weightingPotential = nesse.potentialFromH5(WP_filename, rotate90=True)
print("WP:")
print(total_size(weightingPotential))
# print(tracemalloc.get_traced_memory())

print("Presim:")
snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

sim = nesse.Simulation("Example_sim", 125, Efield, _weightingPotential=weightingPotential, contacts=1)

spiceFile= "g:/nesse/config/Spice/spice_step_New_1ns.csv"
sim.setElectronicResponse(spiceFile)

sim.setIDP(lambda x, y, z : float(ID) * 1e16)

ef_bounds = [[axis[0],axis[-1]] for axis in Efield.grid]
bounds = np.stack((ef_bounds[0],ef_bounds[1],[0,0.002]))
sim.setBounds(bounds)

print("Sim:")
snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

g:\nesse\tests
WP:
3.83984375
Presim:
Top 10 lines
#1: g:\nesse\tests/../src\nesse\field.py:99: 64294.0 KiB
    fieldMag_interp = Interp3D(fieldMag.astype('double',order="C"), x,y,z)
#2: g:\nesse\tests/../src\nesse\field.py:96: 64293.6 KiB
    fieldz_interp = Interp3D(self.fieldz.astype('double',order="C"), x,y,z)
#3: g:\nesse\tests/../src\nesse\field.py:95: 64293.6 KiB
    fieldy_interp = Interp3D(self.fieldy.astype('double',order="C"), x,y,z)
#4: g:\nesse\tests/../src\nesse\field.py:94: 64293.6 KiB
    fieldx_interp = Interp3D(self.fieldx.astype('double', order="C"), x,y,z)
#5: C:\Users\RJ\AppData\Local\Temp\ipykernel_12780\2068151057.py:1: 55569.6 KiB
    Is = [qp.q*np.dot(qp.vel,
#6: g:\nesse\tests/../src\nesse\field.py:151: 39734.5 KiB
    Ez = np.rot90(np.array(f["Ez"]),axes=(0,2))
#7: g:\nesse\tests/../src\nesse\field.py:150: 39734.5 KiB
    Ey = np.rot90(np.array(f["Ey"]),axes=(0,2))
#8: g:\nesse\tests/../src\nesse\field.py:149: 39734.5 KiB
    Ex = np.rot90(np.array(f["Ex"]),a

In [63]:
print("WP:")
print(total_size(weightingPotential))
print(len(weightingPotential.data))

weightingField = weightingPotential.toField()

print("WF:")
print(total_size(weightingField))

print(len(weightingField.fieldx))


WP:
3.83984375
370
WF:
48224.1552734375
370


In [50]:
weightingPotential.data[0][0]

array([7.1045565e-07, 7.1942839e-07, 7.2015280e-07, 7.2046311e-07,
       7.2140023e-07, 7.2286082e-07, 7.2371790e-07, 7.2509556e-07,
       7.2610800e-07, 7.2740863e-07, 7.2785974e-07, 7.2858575e-07,
       7.2844864e-07, 7.2857245e-07, 7.2782109e-07, 7.2731564e-07,
       7.2592474e-07, 7.2476382e-07, 7.2270956e-07, 7.2086920e-07,
       7.1882948e-07, 7.1724583e-07, 7.1501756e-07, 7.1323456e-07,
       7.1282471e-07, 7.1241516e-07, 7.1079830e-07, 7.0902610e-07,
       7.0604693e-07, 7.0360528e-07, 7.0026971e-07, 6.9703378e-07,
       6.9291309e-07], dtype=float32)

In [56]:
weightingField.fieldz[0][0]

array([ 2.01886669e-05,  1.17213349e-05,  3.04952264e-05,  2.89017917e-05,
        1.43861398e-05,  1.39060430e-05,  1.34087168e-05,  1.33519061e-05,
        1.04086939e-05,  7.88271427e-06,  5.29712997e-06,  2.65007839e-06,
       -5.98374754e-08, -2.82400288e-06, -5.65568916e-06, -8.53324309e-06,
       -1.14829745e-05, -1.44680962e-05, -1.75256282e-05, -2.10851431e-05,
       -2.17403285e-05, -2.28714198e-05, -2.40677036e-05, -3.88920307e-05,
       -3.83875740e-05, -7.42892735e-06, -1.24243088e-05, -1.74180605e-05,
       -2.19158828e-05, -2.59974040e-05, -2.95718201e-05, -3.31050251e-05,
       -3.70865964e-05], dtype=float32)

In [62]:
weightingField.grid[2]

array([-1.0000000e-03, -5.5555598e-04, -9.9999997e-06,  0.0000000e+00,
        8.3333252e-05,  1.6666650e-04,  2.4999975e-04,  3.3333301e-04,
        4.4444425e-04,  5.5555551e-04,  6.6666678e-04,  7.7777798e-04,
        8.8888849e-04,  9.9999900e-04,  1.1111095e-03,  1.2222200e-03,
        1.3333325e-03,  1.4444450e-03,  1.5555575e-03,  1.6666700e-03,
        1.7500025e-03,  1.8333350e-03,  1.9166676e-03,  2.0000001e-03,
        2.0099999e-03,  2.1463900e-03,  2.2827801e-03,  2.4191700e-03,
        2.5555601e-03,  2.6666699e-03,  2.7777799e-03,  2.8888900e-03,
        3.0000000e-03], dtype=float32)

In [82]:
print(total_size(weightingField.fieldx))
print(total_size(weightingPotential.data))

print(np.shape(weightingField.fieldx))
print(np.shape(weightingPotential.data))

16073.4609375
0.140625
(370, 337, 33)
(370, 337, 33)


In [103]:
weightingFieldx_interp, weightingFieldy_interp, weightingFieldz_interp, weightingFieldMag_interp = sim.weightingPotential[0].toField().interpolate(True)

In [107]:
print(total_size(weightingFieldx_interp))
print(total_size(weightingFieldy_interp))
print(total_size(weightingFieldz_interp))
print(total_size(weightingFieldMag_interp))

16080.43359375
16080.43359375
16080.43359375
16080.43359375


In [106]:
total_size(sim.weightingPotential[0].toField().interpolate(True))

64311.0703125

In [92]:
print(total_size(weightingFieldx_interp.x))

1.5546875


In [105]:
weightingFieldy_interp.v

array([[[ 0.00037081,  0.00038963,  0.00041777, ...,  0.00050103,
          0.00049796,  0.0004994 ],
        [ 0.00037192,  0.00039366,  0.00041915, ...,  0.00050325,
          0.00050273,  0.00050165],
        [ 0.00037743,  0.00039843,  0.00042552, ...,  0.00051111,
          0.00050956,  0.00050948],
        ...,
        [-0.00037985, -0.00040088, -0.00042794, ..., -0.00051339,
         -0.00051184, -0.00051174],
        [-0.00037446, -0.00039621, -0.00042168, ..., -0.00050564,
         -0.00050511, -0.00050402],
        [-0.00037338, -0.00039226, -0.00042034, ..., -0.00050346,
         -0.00050042, -0.00050181]],

       [[ 0.00040446,  0.0004385 ,  0.0004744 , ...,  0.00062005,
          0.00062306,  0.00062025],
        [ 0.00040849,  0.00043992,  0.00047908, ...,  0.00062578,
          0.00062615,  0.00062595],
        [ 0.00041316,  0.00044625,  0.00048493, ...,  0.00063457,
          0.00063602,  0.0006348 ],
        ...,
        [-0.00041569, -0.00044879, -0.00048747, ..., -

In [90]:
weightingFieldx_interp.v[0][0]

array([0.00026173, 0.00027911, 0.00030521, 0.0003028 , 0.00030942,
       0.00031032, 0.00031692, 0.00031778, 0.00032554, 0.00032747,
       0.00033502, 0.0003367 , 0.00034398, 0.00034536, 0.0003523 ,
       0.00035333, 0.00035985, 0.00036048, 0.00036653, 0.00036669,
       0.00037155, 0.00037069, 0.00037525, 0.00037413, 0.00037709,
       0.00037723, 0.00038227, 0.00038142, 0.00038525, 0.00038324,
       0.00038591, 0.00038312, 0.00038478])

In [85]:
print(type(weightingField.fieldx))
print(type(weightingPotential.data))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [76]:
370*337*33*32 / 8000

16459.08

In [74]:
for i in range(len(weightingField.fieldx)):
    if total_size(weightingField.fieldx[i]) != total_size(weightingPotential.data[i]):
        print(i)
        print(total_size(weightingField.fieldx[i]))
        print(total_size(weightingPotential.data[i]))

        print(np.shape(weightingField.fieldx[i]))
        print(np.shape(weightingPotential.data[i]))

In [40]:
sim.simulate(Events, ds=1e-6, diffusion=False, dt=1e-10, maxPairs=1, silence=True, parallel=False)

print("Postsim:")
snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

print("-----------------------------------")
print(f"Events size: {total_size(Events)}")
print(f"Event size: {total_size(Events[0])}")
print(f"Quasiparticles size: {total_size(Events[0].quasiparticles)}")
print(f"Quasiparticle size: {total_size(Events[0].quasiparticles[0])}")
print(f"Quasiparticle pos size: {total_size(Events[0].quasiparticles[0].pos)}")

  0%|          | 0/1 [00:00<?, ?it/s]

Postsim:
Top 10 lines
#1: g:\nesse\tests/../src\nesse\field.py:99: 96440.8 KiB
    fieldMag_interp = Interp3D(fieldMag.astype('double',order="C"), x,y,z)
#2: g:\nesse\tests/../src\nesse\field.py:96: 96440.4 KiB
    fieldz_interp = Interp3D(self.fieldz.astype('double',order="C"), x,y,z)
#3: g:\nesse\tests/../src\nesse\field.py:95: 96440.4 KiB
    fieldy_interp = Interp3D(self.fieldy.astype('double',order="C"), x,y,z)
#4: g:\nesse\tests/../src\nesse\field.py:94: 96440.4 KiB
    fieldx_interp = Interp3D(self.fieldx.astype('double', order="C"), x,y,z)
#5: C:\Users\RJ\AppData\Local\Temp\ipykernel_12780\2068151057.py:1: 55569.6 KiB
    Is = [qp.q*np.dot(qp.vel,
#6: g:\nesse\tests/../src\nesse\field.py:151: 19867.3 KiB
    Ez = np.rot90(np.array(f["Ez"]),axes=(0,2))
#7: g:\nesse\tests/../src\nesse\field.py:150: 19867.3 KiB
    Ey = np.rot90(np.array(f["Ey"]),axes=(0,2))
#8: g:\nesse\tests/../src\nesse\field.py:149: 19867.3 KiB
    Ex = np.rot90(np.array(f["Ex"]),axes=(0,2))
#9: g:\nesse\tests

In [5]:
event = Events[0]
qp = event.quasiparticles[0]

In [6]:
qp.pos

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e-06],
       [1.1637479e-13, 1.3584952e-14, 2.0000000e-06],
       ...,
       [1.4444829e-07, 4.1040304e-09, 2.0000001e-03],
       [1.4444829e-07, 4.1040304e-09, 2.0000001e-03],
       [1.4444829e-07, 4.1040304e-09, 2.0000001e-03]], dtype=float32)

In [7]:
weightingFieldx_interp, weightingFieldy_interp, weightingFieldz_interp, weightingFieldMag_interp = sim.weightingPotential[0].toField().interpolate(True)

In [8]:
I = [qp.q*np.dot(qp.vel[j],
                        [weightingFieldx_interp(qp.pos[j]),
                        weightingFieldy_interp(qp.pos[j]),
                        weightingFieldz_interp(qp.pos[j])]) for j in range(len(qp.pos))]

In [17]:
np.array((weightingFieldx_interp(qp.pos),
                        weightingFieldy_interp(qp.pos),
                        weightingFieldz_interp(qp.pos))).T

array([[ 0.00000000e+00,  0.00000000e+00,  5.33306656e+01],
       [ 2.78320574e-05, -8.23975384e-06,  5.86639634e+01],
       [ 5.56641144e-05, -1.64795078e-05,  6.39972611e+01],
       ...,
       [ 0.00000000e+00,  0.00000000e+00, -1.71828133e+02],
       [ 0.00000000e+00,  0.00000000e+00, -1.71828133e+02],
       [ 0.00000000e+00,  0.00000000e+00, -1.71828133e+02]])

In [19]:
qp.vel

array([[-0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00, -0.0000000e+00,  5.8222199e+04],
       [ 6.7747869e-03,  7.9085131e-04,  5.8215246e+04],
       ...,
       [ 9.2403440e-07,  1.0068612e-08,  1.8148800e-02],
       [ 9.0705333e-07,  9.8835793e-09,  1.7815277e-02],
       [ 8.9038429e-07,  9.7019477e-09,  1.7487884e-02]], dtype=float32)

In [32]:
Is = qp.q * np.sum(qp.vel * np.array((weightingFieldx_interp(qp.pos),
                            weightingFieldy_interp(qp.pos),
                            weightingFieldz_interp(qp.pos))).T, axis=1)

In [34]:
I-Is

array([0., 0., 0., ..., 0., 0., 0.])

In [26]:
np.array(I)

array([-0.00000000e+00, -3.82541036e-12, -4.17269026e-12, ...,
        3.49269142e-18,  3.42850568e-18,  3.36549963e-18])

In [33]:
Is

array([-0.00000000e+00, -3.82541036e-12, -4.17269026e-12, ...,
        3.49269142e-18,  3.42850568e-18,  3.36549963e-18])

In [18]:
qp.pos

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e-06],
       [1.1637479e-13, 1.3584952e-14, 2.0000000e-06],
       ...,
       [1.4444829e-07, 4.1040304e-09, 2.0000001e-03],
       [1.4444829e-07, 4.1040304e-09, 2.0000001e-03],
       [1.4444829e-07, 4.1040304e-09, 2.0000001e-03]], dtype=float32)

In [ ]:
nesse.find_first(qp.pos, sim.electricField.grid[0])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [50]:
weightingFieldx_interp(qp.pos)

array([0.00000000e+00, 2.78320574e-05, 5.56641144e-05, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [35]:
I = [qp.q*np.dot(qp.vel,
                        [weightingFieldx_interp(qp.pos),
                        weightingFieldy_interp(qp.pos),
                        weightingFieldz_interp(qp.pos)])]

[autoreload of nesse.interp_3d failed: Traceback (most recent call last):
  File "c:\Users\RJ\anaconda3\envs\nesse_dev\lib\site-packages\IPython\extensions\autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\RJ\anaconda3\envs\nesse_dev\lib\site-packages\IPython\extensions\autoreload.py", line 471, in superreload
    module = reload(module)
  File "c:\Users\RJ\anaconda3\envs\nesse_dev\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "g:\nesse\tests/../src\nesse\interp_3d.py", line 11, in <module>
    def find_first(item, vec):
  File "c:\Users\RJ\anaconda3\envs\nesse_dev\lib\site-packages\numba\np\ufunc\decorators.py", line 203, in wrap
    guvec.add(fty)
  File "c:\Users\RJ\anaconda3\envs\ness

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [42]:
t0 = time.time()
sim.calculateInducedCurrent(Events, 1e-9, detailed=True)
t1 = time.time()
print(t1-t0)
# print("Induced Current:")
# print(tracemalloc.get_traced_memory())

print(f"Events size: {total_size(Events)}")
print(f"Event size: {total_size(Events[0])}")

print(type(Events[0].dI[0]))
print(type(Events[0].dt[0]))

print("-----------------------------------")

print("PostCurrent:")
snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

1.2149314880371094
Events size: 3555.50390625
Event size: 3555.44140625
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
-----------------------------------
PostCurrent:
Top 10 lines
#1: g:\nesse\tests/../src\nesse\field.py:99: 64294.0 KiB
    fieldMag_interp = Interp3D(fieldMag.astype('double',order="C"), x,y,z)
#2: g:\nesse\tests/../src\nesse\field.py:96: 64293.6 KiB
    fieldz_interp = Interp3D(self.fieldz.astype('double',order="C"), x,y,z)
#3: g:\nesse\tests/../src\nesse\field.py:95: 64293.6 KiB
    fieldy_interp = Interp3D(self.fieldy.astype('double',order="C"), x,y,z)
#4: g:\nesse\tests/../src\nesse\field.py:94: 64293.6 KiB
    fieldx_interp = Interp3D(self.fieldx.astype('double', order="C"), x,y,z)
#5: C:\Users\RJ\AppData\Local\Temp\ipykernel_12780\2068151057.py:1: 55569.6 KiB
    Is = [qp.q*np.dot(qp.vel,
#6: g:\nesse\tests/../src\nesse\field.py:151: 19867.3 KiB
    Ez = np.rot90(np.array(f["Ez"]),axes=(0,2))
#7: g:\nesse\tests/../src\nesse\field.py:150: 19867.3 KiB
    Ey = np.

In [34]:
Events[0].dt[0]

array([0.00e+00, 1.00e-09, 2.00e-09, 3.00e-09, 4.00e-09, 5.00e-09,
       6.00e-09, 7.00e-09, 8.00e-09, 9.00e-09, 1.00e-08, 1.10e-08,
       1.20e-08, 1.30e-08, 1.40e-08, 1.50e-08, 1.60e-08, 1.70e-08,
       1.80e-08, 1.90e-08, 2.00e-08, 2.10e-08, 2.20e-08, 2.30e-08,
       2.40e-08, 2.50e-08, 2.60e-08, 2.70e-08, 2.80e-08, 2.90e-08,
       3.00e-08, 3.10e-08, 3.20e-08, 3.30e-08, 3.40e-08, 3.50e-08,
       3.60e-08, 3.70e-08, 3.80e-08, 3.90e-08, 4.00e-08, 4.10e-08,
       4.20e-08, 4.30e-08, 4.40e-08, 4.50e-08, 4.60e-08, 4.70e-08,
       4.80e-08, 4.90e-08, 5.00e-08, 5.10e-08, 5.20e-08, 5.30e-08,
       5.40e-08, 5.50e-08, 5.60e-08, 5.70e-08, 5.80e-08, 5.90e-08,
       6.00e-08, 6.10e-08, 6.20e-08, 6.30e-08, 6.40e-08, 6.50e-08,
       6.60e-08, 6.70e-08, 6.80e-08, 6.90e-08, 7.00e-08, 7.10e-08,
       7.20e-08, 7.30e-08, 7.40e-08, 7.50e-08, 7.60e-08, 7.70e-08,
       7.80e-08, 7.90e-08, 8.00e-08, 8.10e-08, 8.20e-08, 8.30e-08,
       8.40e-08, 8.50e-08, 8.60e-08, 8.70e-08, 8.80e-08, 8.90e